# Surface Codes with `deltakit` and `crumpy`

## Using `deltakit` with `crumpy`

Let's generate and visualize a `RotatedPlanarCode` with `deltakit.explorer.codes`:

In [ ]:
from deltakit.explorer.codes import RotatedPlanarCode, UnrotatedPlanarCode
import ipywidgets

def getPlanarCode(is_rotated: bool, w: int, h: int) -> RotatedPlanarCode | UnrotatedPlanarCode:
    return RotatedPlanarCode(width=w, height=h) if is_rotated else UnrotatedPlanarCode(width=w, height=h)

def savePlanarCodeImage(code: RotatedPlanarCode | UnrotatedPlanarCode) -> ipywidgets.Image:
    filename = f"./images/{'u' if isinstance(code, UnrotatedPlanarCode) else ''}rpc_{code.width}x{code.height}.png"
    code.draw_patch(filename)

    file = open(filename, "rb")
    image = file.read()
    image_widg = ipywidgets.Image(
        value=image,
        format='png',
        width=500
    )
    return image_widg

code = getPlanarCode(is_rotated=True, w=2, h=2)
image_widg = savePlanarCodeImage(code)
image_widg

We can create a memory experiment circuit from our generated planar code:

In [ ]:
from deltakit.explorer.codes import css_code_memory_circuit
from deltakit.circuit.gates import PauliBasis

deltakit_circuit = css_code_memory_circuit(code, num_rounds=1, logical_basis=PauliBasis.Z)
deltakit_circuit

To use this circuit with `crumpy`'s `CircuitWidget`, we need to convert to a `stim` circuit:

In [ ]:
stim_circuit = deltakit_circuit.as_stim_circuit()
stim_circuit

This circuit can be displayed directly with `CircuitWidget.from_stim`:

In [ ]:
from crumpy import CircuitWidget

CircuitWidget.from_stim(stim_circuit)

Putting it all together in a function:

In [ ]:
import stim

def stimSurfaceCode(is_rotated: bool, w: int, h: int) -> stim.Circuit:
    code = getPlanarCode(is_rotated, w, h)
    circuit = css_code_memory_circuit(code, num_rounds=1, logical_basis=PauliBasis.Z)
    return circuit.as_stim_circuit()

stim_circuit = stimSurfaceCode(True, 3, 3)
circuit_widg = CircuitWidget.from_stim(stim_circuit)
circuit_widg

Let's make things more interactive with some control widgets (from `ipywidgets`). We can create sliders that vary the width and height of the planar code we generate, and a checkbox for `UnrotatedPlanarCode` or `RotatedPlanarCode`:

In [ ]:
# Some helper functions for creating sliders and checkboxes (we'll resuse these later)
def WSlider():
    return ipywidgets.IntSlider(description="Width", value=3, min=2, max=5)
def HSlider():
    return ipywidgets.IntSlider(description="Height", value=3, min=2, max=5)
def Checkbox():
    return ipywidgets.Checkbox(
        value=True,
        description='Rotated?',
        disabled=False,
    )

# Create widgets
w_slider = WSlider()
h_slider = HSlider()
checkbox = Checkbox()
dynamic_circuit = CircuitWidget()

dynamic_circuit_callback = lambda _: str(stimSurfaceCode(checkbox.value, w_slider.value, h_slider.value))

# Link control widgets to the circuit
ipywidgets.dlink((w_slider, "value"), (dynamic_circuit, "stim"), dynamic_circuit_callback)
ipywidgets.dlink((h_slider, "value"), (dynamic_circuit, "stim"), dynamic_circuit_callback)
ipywidgets.dlink((checkbox, "value"), (dynamic_circuit, "stim"), dynamic_circuit_callback)

display(checkbox, w_slider, h_slider, dynamic_circuit)

## Pauli Propagation

Let's explore another feature of `crumpy` and add a Pauli marker to a circuit:

In [ ]:
stim_str_with_pauli = "#!pragma MARKX(0) 0\n" + str(stimSurfaceCode(True, 2, 2))

CircuitWidget(stim=stim_str_with_pauli)

Let's add some interactivity with a slider (to change which line the error is on) and some radio buttons (to select the error type):

In [ ]:
# Error slider and radio buttons
def PauliSlider():
    return ipywidgets.IntSlider(value=0, min=0, readout=False, orientation='vertical', layout=ipywidgets.Layout(height='120px'))
def PauliSelect():
    return ipywidgets.RadioButtons(
        options=['X', 'Y', 'Z', 'none'],
        default='X',
        description='Pauli error:',
        disabled=False
    )

# Returns a stim circuit with a specified Pauli marker added at the front
def circuitWithError(stim_circuit: stim.Circuit, err: str, idx: int) -> str:
    coords = list(stim_circuit.get_final_qubit_coordinates().items())
    
    # Sort qubits by coords in circuit line order (bottom to top on viz, for easy slider use)
    coords.sort(key=lambda pair: (-pair[1][1], -pair[1][0]))
    
    return f"""
    {'' if err == 'none' else f'#!pragma MARK{err}(0) {coords[min(idx, len(coords) - 1)][0]}'}
    {stim_circuit}
    """

# Update a slider's max value and color based on given Pauli err and num_lines in the circuit
def updatePauliSlider(slider: ipywidgets.IntSlider, err: str, num_lines: int) -> None:
    legend = {
        'X':'red',
        'Y':'green',
        'Z':'blue',
        'none':'black'
    }
    slider.style.handle_color = legend[err]

    slider.max = num_lines - 1

# Returns a 2x2 rotated surface code (as a stim str), with the given Pauli err on the idx-th circuit line
def updateErrorCircuit(slider: ipywidgets.IntSlider, err: str, idx: int) -> str:
    stim_circuit = stimSurfaceCode(True, 2, 2)
    updatePauliSlider(slider, err, len(stim_circuit.get_final_qubit_coordinates()))
    return circuitWithError(stim_circuit, err, idx)


pauli_slider = PauliSlider()
pauli_select = PauliSelect()
circuit_with_error = CircuitWidget()

circuit_with_error_callback = lambda _: updateErrorCircuit(pauli_slider, pauli_select.value, pauli_slider.value)

# Link error slider and selector to circuit
ipywidgets.dlink((pauli_select, 'value'), (circuit_with_error, 'stim'), circuit_with_error_callback)
ipywidgets.dlink((pauli_slider, 'value'), (circuit_with_error, 'stim'), circuit_with_error_callback)

ipywidgets.HBox([circuit_with_error, pauli_slider, pauli_select])


## Exploring Surface Codes

Let's combine everything we've done so far into one interactive output with surface code controls *and* Pauli error controls:

In [ ]:
# Creates a surface code circuit with is_rotated, width, and height parameters that has the
# given Pauli err at qubit idx
def finalCircuit(is_rotated: bool, w: int, h: int, err: str, idx: int) -> str:
    stim_circuit = stimSurfaceCode(is_rotated, w, h)
    updatePauliSlider(final_pauli_slider, err, len(stim_circuit.get_final_qubit_coordinates()))
    return circuitWithError(stim_circuit, err, idx)

final_circuit = CircuitWidget()
final_pauli_slider = PauliSlider()
final_pauli_select = PauliSelect()
final_w_slider = WSlider()
final_h_slider = HSlider()
final_checkbox = Checkbox()

final_callback = lambda _: finalCircuit(is_rotated=final_checkbox.value, w=final_w_slider.value, h=final_h_slider.value, err=final_pauli_select.value, idx=final_pauli_slider.value)

ipywidgets.dlink((final_pauli_select, 'value'), (final_circuit, 'stim'), final_callback)
ipywidgets.dlink((final_pauli_slider, 'value'), (final_circuit, 'stim'), final_callback)
ipywidgets.dlink((final_w_slider, 'value'), (final_circuit, 'stim'), final_callback)
ipywidgets.dlink((final_h_slider, 'value'), (final_circuit, 'stim'), final_callback)
ipywidgets.dlink((final_checkbox, 'value'), (final_circuit, 'stim'), final_callback)

display(ipywidgets.Label("Planar code:"), final_checkbox, final_w_slider, final_h_slider, ipywidgets.HBox([final_circuit, final_pauli_slider, final_pauli_select]))

Let's explore another interactive visualization. `draw_patch` will visualize the surface code patches of a given planar code :

In [ ]:
from deltakit.explorer.codes import RotatedPlanarCode

from snippet import draw_patch

rp_code = RotatedPlanarCode(width=5, height=5)
draw_patch(rp_code)

Combining this with `CircuitWidget`, we can create an clickable surface code patch with a linked circuit visualization. Clicking a qubit on the interactive graph changes the Pauli error present at that qubit on the circuit diagram:

In [ ]:
from deltakit.explorer.codes import css_code_memory_circuit, RotatedPlanarCode, UnrotatedPlanarCode
from deltakit.circuit import GateLayer, Circuit
from deltakit.circuit.gates import PauliBasis
import ipywidgets
from plotly.callbacks import Points
import stim

from snippet import draw_patch
from crumpy import CircuitWidget

def interactivePlanarCode(planar_code: RotatedPlanarCode | UnrotatedPlanarCode) -> ipywidgets.widget:
  # Returns the qubit at the specifed qubit coord, -1 if not found
  def lookupQubitFromCoord(stim_circuit: stim.Circuit, coord: list[int, int]) -> int:
    coords = list(stim_circuit.get_final_qubit_coordinates().items())
    for idx, pair in coords:
      if coord == pair:
        return idx
    return -1

  # Takes a map of qubit coords to Pauli errors and applies them at the start of the given stim circuit.
  # Returns the resulting stim circuit (str)
  def circuitWithPaulisAt(stim_circuit: stim.Circuit, active_paulis: dict[tuple[int, int], str]) -> str:
    stim_str = str(stim_circuit)

    err_circuit = ''.join([
      f"#!pragma MARK{pauli.upper()}(0) {lookupQubitFromCoord(stim_circuit, list(coord))}\n"for coord, pauli in active_paulis.items()
    ]) + stim_str
    return err_circuit

  # Given a circuit, CircuitWidget, dict of active paulis, and a new point, toggle the Pauli error at that point.
  # Updates the given circuit_widg to reflect the added/removed Pauli.
  def updateCicruitPaulis(points: Points, original_circuit: stim.Circuit, circuit_widg: CircuitWidget, active_paulis: dict[tuple[int, int], str]) -> None:
    xs = points.xs
    ys = points.ys

    if not xs or not ys or len(xs) != len(ys):
      return
    
    for point in zip(xs, ys):
      if point in active_paulis:
        if active_paulis[point] == 'Z':
          del active_paulis[point]
        else:
          active_paulis[point] = chr(ord(active_paulis[point]) + 1)
      else:
        active_paulis[point] = 'X'

    circuit_widg.stim = circuitWithPaulisAt(original_circuit, active_paulis)

  # Create planar code memory circuit
  deltakit_circuit = css_code_memory_circuit(
      css_code=planar_code,
      num_rounds=1,
      logical_basis=PauliBasis.Z,
  )
  deltakit_circuit = Circuit([layer for layer in deltakit_circuit.layers if isinstance(layer, GateLayer)]) # strip detectors/observables

  # Create CircuitWidget from the circuit
  stim_circuit = deltakit_circuit.as_stim_circuit()

  circuit_widg = CircuitWidget.from_stim(stim_circuit)

  circuit_widg.layout.max_height = "650px"
  circuit_widg.layout.max_width = "650px"
  circuit_widg.layout.overflow = "auto"

  # Register click callback that will update the Pauli markers on the CircuitWidget
  active_paulis = dict()
  qubitClickCallback = lambda points: updateCicruitPaulis(points, stim_circuit, circuit_widg, active_paulis)

  plot_widg = draw_patch(planar_code, qubitClickCallback, qubitClickCallback, unrotated_code=isinstance(planar_code, UnrotatedPlanarCode))
  return ipywidgets.HBox([plot_widg, circuit_widg], layout=ipywidgets.Layout(justify_content='center', align_items='flex-start', width='auto'))

planar_code = RotatedPlanarCode(width=3, height=3)
interactivePlanarCode(planar_code)

We can further add the width slider, height slider, and rotation checkbox used previous to vary the planar code parameters:

In [ ]:
w_slider_interact = WSlider()
w_slider_interact.continuous_update = False
h_slider_interact = HSlider()
h_slider_interact.continuous_update = False
checkbox_interact = Checkbox()

plot_output = ipywidgets.Output()

def update_plot(change=None):
    plot_output.clear_output(wait=True)
    planar_code = getPlanarCode(is_rotated=checkbox_interact.value, w=w_slider_interact.value, h=h_slider_interact.value)
    fig = interactivePlanarCode(planar_code)
    with plot_output:
        display(fig)


w_slider_interact.observe(update_plot, names='value')
h_slider_interact.observe(update_plot, names='value')
checkbox_interact.observe(update_plot, names='value')

update_plot()

ipywidgets.VBox([checkbox_interact, w_slider_interact, h_slider_interact, plot_output], layout=ipywidgets.Layout(align_items='center'))